In [55]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import plotly.express as px
import plotly.graph_objects as go

## NBA ELO

In [2]:
elo_df = pd.read_csv("datasets/nba-forecasts/nba_elo.csv")
elo_df.head(5)

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carm-elo1_pre,carm-elo2_pre,carm-elo_prob1,carm-elo_prob2,carm-elo1_post,carm-elo2_post,raptor1_pre,raptor2_pre,raptor_prob1,raptor_prob2,score1,score2,quality,importance,total_rating
0,1946-11-01,1947,0,NaN,TRH,NYK,1300.0,1300.0000,0.640065,0.359935,1293.2767,1306.7233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,68.0,0,NaN,NaN
1,1946-11-02,1947,0,NaN,PRO,BOS,1300.0,1300.0000,0.640065,0.359935,1305.1542,1294.8458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,53.0,0,NaN,NaN
2,1946-11-02,1947,0,NaN,DTF,WSC,1300.0,1300.0000,0.640065,0.359935,1279.6189,1320.3811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,50.0,0,NaN,NaN
3,1946-11-02,1947,0,NaN,STB,PIT,1300.0,1300.0000,0.640065,0.359935,1304.6908,1295.3092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,51.0,0,NaN,NaN
4,1946-11-02,1947,0,NaN,CHS,NYK,1300.0,1306.7233,0.631101,0.368899,1309.6521,1297.0712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,47.0,0,NaN,NaN


In [3]:
print('Number of null values by column\n')
for col in elo_df.columns:
    print(f"{col}:{' ' * (15 - len(col))}{elo_df[col].isnull().sum()}")

Number of null values by column

date:           0
season:         0
neutral:        0
playoff:        68643
team1:          0
team2:          0
elo1_pre:       0
elo2_pre:       0
elo_prob1:      0
elo_prob2:      0
elo1_post:      55
elo2_post:      55
carm-elo1_pre:  68024
carm-elo2_pre:  68024
carm-elo_prob1: 68024
carm-elo_prob2: 68024
carm-elo1_post: 68024
carm-elo2_post: 68024
raptor1_pre:    67094
raptor2_pre:    67094
raptor_prob1:   67094
raptor_prob2:   67094
score1:         55
score2:         55
quality:        0
importance:     70720
total_rating:   70720


_Columns we are interested in are those starting with `elo` and `team1/team2` (and potentially also `score1/score2`). Of these, only the `elo_post` and `score` columns have any missing values. Looking at these rows however, its clear that they actually come from future games which haven't happened yet (from when the dataset is downloaded), and thus it wouldn't be possible to calculate these columns yet anyway as they rely on the outcome of the game. By the time we do the real visualisation, these games will have already occurred so this won't be a problem, but even so we would remove these rows anyway since they are not informative for our visualisations._

In [4]:
print(f"Before filtering: {elo_df.shape}")

# remove future games without scores
elo_df = elo_df[elo_df['score1'].notnull()]

# retain only columns of interest
cols_to_keep = ['season', 'date', 'team1', 'team2', 'playoff', 'elo1_pre', 'elo2_pre', 'elo_prob1', 'elo_prob2', 
                'elo1_post', 'elo2_post', 'score1', 'score2']
filtered_elo_df = elo_df[cols_to_keep]

print(f"After filtering: {filtered_elo_df.shape}")

filtered_elo_df.head(5)

Before filtering: (73273, 27)
After filtering: (73218, 13)


,season,date,team1,team2,playoff,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2
0,1947,1946-11-01,TRH,NYK,NaN,1300.0,1300.0000,0.640065,0.359935,1293.2767,1306.7233,66.0,68.0
1,1947,1946-11-02,PRO,BOS,NaN,1300.0,1300.0000,0.640065,0.359935,1305.1542,1294.8458,59.0,53.0
2,1947,1946-11-02,DTF,WSC,NaN,1300.0,1300.0000,0.640065,0.359935,1279.6189,1320.3811,33.0,50.0
3,1947,1946-11-02,STB,PIT,NaN,1300.0,1300.0000,0.640065,0.359935,1304.6908,1295.3092,56.0,51.0
4,1947,1946-11-02,CHS,NYK,NaN,1300.0,1306.7233,0.631101,0.368899,1309.6521,1297.0712,63.0,47.0


In [32]:
# duplicate the df by switching team1 and team 2, then concatenate this to the original df
reshuffled_cols = list(filtered_elo_df.columns)
for i, c in enumerate(reshuffled_cols):
    if '1' in c:
        reshuffled_cols[i] = c.replace('1', '2')
    elif '2' in c:
        reshuffled_cols[i] = c.replace('2', '1')
        
tmp_switched_df = filtered_elo_df.copy()
tmp_switched_df.columns = reshuffled_cols

expanded_elo_df = pd.concat([filtered_elo_df, tmp_switched_df]).sort_values(by='date')

# rename the columns so one team is the main 'team' and the other is the 'opponent'
expanded_elo_df.columns = [c.replace('1', "") for c in expanded_elo_df.columns]
expanded_elo_df.columns = [c.replace('2', "_opponent") for c in expanded_elo_df.columns]

expanded_elo_df['win'] = expanded_elo_df['score'] > expanded_elo_df['score_opponent']

expanded_elo_df.loc[list(range(3))]

,season,date,team,team_opponent,playoff,elo_pre,elo_opponent_pre,elo_prob,elo_prob_opponent,elo_post,elo_opponent_post,score,score_opponent,win
0,1947,1946-11-01,TRH,NYK,NaN,1300.0,1300.0,0.640065,0.359935,1293.2767,1306.7233,66.0,68.0,False
0,1947,1946-11-01,NYK,TRH,NaN,1300.0,1300.0,0.359935,0.640065,1306.7233,1293.2767,68.0,66.0,True
1,1947,1946-11-02,BOS,PRO,NaN,1300.0,1300.0,0.359935,0.640065,1294.8458,1305.1542,53.0,59.0,False
1,1947,1946-11-02,PRO,BOS,NaN,1300.0,1300.0,0.640065,0.359935,1305.1542,1294.8458,59.0,53.0,True
2,1947,1946-11-02,WSC,DTF,NaN,1300.0,1300.0,0.359935,0.640065,1320.3811,1279.6189,50.0,33.0,True
2,1947,1946-11-02,DTF,WSC,NaN,1300.0,1300.0,0.640065,0.359935,1279.6189,1320.3811,33.0,50.0,False


In [33]:
# save proccessed df as csv
#expanded_elo_df.to_csv("datasets/nba-forecasts/nba_elo_by_team.csv")

### Elo progression over a season

In [29]:
season = 2022
playoff_teams_only = True

season_elos = expanded_elo_df[expanded_elo_df['season']==season]

if playoff_teams_only:
    playoff_teams = season_elos[season_elos['playoff'].notnull()]['team'].unique()
    print("Playoff teams:")
    print(playoff_teams)
    season_elos = season_elos[season_elos['team'].isin(playoff_teams)]

Playoff teams:
['MIN' 'BRK' 'CLE' 'LAC' 'SAS' 'ATL' 'NOP' 'CHO' 'UTA' 'TOR' 'DEN' 'DAL'
 'MEM' 'PHI' 'GSW' 'CHI' 'PHO' 'BOS' 'MIA' 'MIL']


In [30]:
fig = px.line(season_elos, x='date', y="elo_post", color='team', 
              hover_data=['team_opponent', "elo_pre", "elo_opponent_pre", "score", "score_opponent", "elo_prob"],
              title=f"Elo progression by team for {season} season {'(playoff teams only)' if playoff_teams_only else ''}")
fig.show()

### Number of 'unlikely' wins in a season

Look at the elo probabilities of each game to determine how many games a team won vs lost which they were forecasted to win

In [51]:
# threshold for where a team is considered 'likely' to win based on elo probability
likely_win_pct = 0.65

expanded_elo_df['unlikely_win'] = expanded_elo_df['win'] & (expanded_elo_df['elo_prob_opponent'] > likely_win_pct)
expanded_elo_df['unlikely_loss'] = (expanded_elo_df['win'] == False) & (expanded_elo_df['elo_prob'] > likely_win_pct)

unlikely_win_summary_df = expanded_elo_df.groupby(['season', 'team']).agg({'unlikely_win' : 'sum', 'unlikely_loss' : 'sum'})
unlikely_win_summary_df['unlikely_loss'] = -unlikely_win_summary_df['unlikely_loss']

In [69]:
season = 2021

season_unlikely_wins = unlikely_win_summary_df.loc[[season]].sort_values(by='unlikely_win', ascending = False)
season_unlikely_wins=season_unlikely_wins.reset_index()
season_unlikely_wins.head(5)

,season,team,unlikely_win,unlikely_loss
0,2021,WAS,14,-2
1,2021,CHI,13,-3
2,2021,SAC,13,-3
3,2021,ATL,11,-3
4,2021,ORL,10,-6


In [68]:
fig = go.Figure()

fig.add_trace(go.Bar(
    y = season_unlikely_wins['unlikely_win'],
    x = season_unlikely_wins['team'],
    name = 'unlikely_win',
    orientation = 'v',
    marker_color = 'green',
))

fig.add_trace(go.Bar(
    y = season_unlikely_wins['unlikely_loss'],
    x = season_unlikely_wins['team'],
    name = 'unlikely_loss',
    orientation = 'v',
    marker_color = 'red',
))

fig.update_layout(
    barmode = 'relative',
    title = f"Unlikely wins vs unlikely losses for each team over the {season} season (forecasted win prob >{likely_win_pct})"
)
fig.show()
